In [1]:
import scipy as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import squarify
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import re
import xgboost
import scipy
from sklearn.model_selection import train_test_split, GridSearchCV
import sklearn
from sklearn.metrics import mean_squared_log_error
import ipdb
from sklearn.linear_model import RidgeCV, Ridge, Lasso

In [2]:
data_train = pd.read_csv("train.tsv", sep='\t', header=0)
data_train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [3]:
data_test = pd.read_csv("test.tsv", sep='\t', header=0)
data_test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [4]:
data = pd.concat([data_train, data_test], 0)
nrow_train = data_train.shape[0]
print(nrow_train)
y  = np.log1p(data_train["price"])
y_train = np.log1p(data_train["price"])

1482535


In [5]:
data['primary_cat'] = data.category_name.str.extract('([^/]+)/[^/]+/[^/]+')
data['secondary_cat'] = data.category_name.str.extract('[^/]+/([^/]+/[^/]+)')

c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [6]:
non_text = [',','-','.','&','%','#', '(',')','[',']','*','!','✨','▪️','"','~',"'", '<', '>', '®','™']
for symbols in non_text: 
    data['primary_cat'] = data['primary_cat'].str.replace(symbols, '') 
    data['secondary_cat'] = data['secondary_cat'].str.replace(symbols, '') 
    data['item_description'] = data['item_description'].str.replace(symbols, '') 
    data['brand_name'] = data['brand_name'].str.replace(symbols, '')
    data['name'] = data['name'].str.replace(symbols, '')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175894 entries, 0 to 693358
Data columns (total 11 columns):
brand_name           object
category_name        object
item_condition_id    int64
item_description     object
name                 object
price                float64
shipping             int64
test_id              float64
train_id             float64
primary_cat          object
secondary_cat        object
dtypes: float64(3), int64(2), object(6)
memory usage: 199.2+ MB


In [7]:
data = data.apply(lambda x: x.astype(str).str.lower())

In [8]:
data.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,primary_cat,secondary_cat
0,nan,men/tops/t-shirts,3,no description yet,mlb cincinnati reds t shirt size xl,10.0,1,nan,0.0,men,tops/tshirts
1,razer,electronics/computers & tablets/components & p...,3,this keyboard is in great condition and works ...,razer blackwidow chroma keyboard,52.0,0,nan,1.0,electronics,computers tablets/components parts
2,target,women/tops & blouses/blouse,1,adorable top with a hint of lace and a key hol...,avaviv blouse,10.0,1,nan,2.0,women,tops blouses/blouse
3,nan,home/home décor/home décor accents,1,new with tags leather horses retail for rm eac...,leather horse statues,35.0,1,nan,3.0,home,home décor/home décor accents
4,nan,women/jewelry/necklaces,1,complete with certificate of authenticity,24k gold plated rose,44.0,0,nan,4.0,women,jewelry/necklaces


In [9]:
features = [data['name'], data['brand_name'] ,data['item_description'] ,data['primary_cat'], data['secondary_cat']]
for x in features:
    x = x.fillna(value = 'none', inplace=False, axis=0)
    print(x.isnull().sum())

0
0
0
0
0


In [10]:
def tokenizer(text):
    if text:
        result = re.findall('[a-z]{2,}', text.lower())
    else:
        result = []
    return result

In [11]:
#combining all text values within one variable X_text
X_text = (data['name'] + ' ' + data['brand_name'] + ' ' +
          data['item_description'] + ' ' + data['primary_cat'] + ' ' + 
          data['secondary_cat']).values

In [12]:
X_text

array([ 'mlb cincinnati reds t shirt size xl nan no description yet men tops/tshirts',
       'razer blackwidow chroma keyboard razer this keyboard is in great condition and works like it came out of the box all of the ports are tested and work perfectly the lights are customizable via the razer synapse app on your pc electronics computers  tablets/components  parts',
       'avaviv blouse target adorable top with a hint of lace and a key hole in the back the pale pink is a 1x and i also have a 3x available in white women tops  blouses/blouse',
       ...,
       'galaxy s8 hard shell case nan new free shipping basstop case electronics cell phones  accessories/cases covers  skins',
       'hi low floral kimono nan floral kimono tropical print open front hi low design great for layering or as a cover up women swimwear/coverups',
       'freeship 2 floral scrub tops medium nan 2 floral scrub tops worn less than 5 times each brown has a belt tie women tops  blouses/tshirts'], dtype=object

In [13]:
#instantiating a TFidVectorizer
count_desc = TfidfVectorizer(tokenizer=tokenizer, max_features = 50000, ngram_range = (1,3),stop_words = "english")

In [14]:
X_text = count_desc.fit_transform(X_text)

In [15]:
X_text

<2175894x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 78711917 stored elements in Compressed Sparse Row format>

In [16]:
#Getting dummies for the non_text values for item_condition with range (1-5) and shipping (0,1)
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(data[[
    "item_condition_id", "shipping"]], sparse = True).values)

In [17]:
#re-combining transformed features into one X value:
X = scipy.sparse.hstack((X_dummies, X_text)).tocsr()

In [18]:
X_training = X[:nrow_train]
X_testing = X[nrow_train:]

In [19]:
print(X_training.shape)
print(y.shape)

(1482535, 50007)
(1482535,)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_training, y, test_size=0.4, random_state=42)

In [21]:
print(X_train.shape)
print(y_train.shape)

(889521, 50007)
(889521,)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge, Lasso

for Model in [Ridge, Lasso]:
    model = Model()
    print('%s: %s' % (Model.__name__,
                      cross_val_score(model, X_train, y_train).mean()))

Ridge: 0.581129232493
Lasso: -1.16485878458e-05


In [ ]:
alphas = np.logspace(-3, -1, 30)

plt.figure(figsize=(5, 3))

for Model in [Lasso, Ridge]:
    scores = [cross_val_score(Model(alpha), X_train, y_train, cv=3).mean()
            for alpha in alphas]
    plt.plot(alphas, scores, label=Model.__name__)

plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.linear_model import Ridge

#SGDRegressor is for when one is adding more data as we go. Linear Regresion would be a better fit for this problem.
#Make sure to regularize. Make sure paramter used to regularize is tested in gridsearch.
#Regularization for regression. Adds penalties
#Ridge is generally used for regularization (method of linear regression to reduce overfitting. Adding term of cost function)
#R^2 Directly related to RMSE but takes out relation to units we're using. Need to pair with normalization of data.
#alpha = regularization term
#Check best_params for gscv

model = Ridge()
params={'alpha': [2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.01]}
gscv = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_mean_squared_log_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)

gscv.fit(X_train, y_train)
#best estimators
print(gscv.best_estimator_)
#printing best scores
print(gscv.best_score_)
#printing best parameters for optimal parameter tuning
print(gscv.best_params_)


#Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None,
#   normalize=False, random_state=None, solver='auto', tol=0.001)
#-0.0153163715869
#{'alpha': 2}

In [ ]:
pd.DataFrame(gscv.cv_results_)

In [ ]:
predictions = gscv.predict(X_test)
#Would need to do a root mean squared error on the predictions vs real y_test.
from sklearn.metrics import mean_squared_error
from math import sqrt

rmsle = sqrt(mean_squared_error(y_test, predictions))

In [ ]:
print(rmsle)

In [ ]:
#The optimal parameteres from GridSearchCV
ridge = Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None, 
              normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
ridge.fit(X_train, y_train)

In [ ]:
predictions = ridge.predict(X_test)

In [ ]:
rmsle = sqrt(mean_squared_error(y_test, predictions))
print(rmsle)

Slightly better! Let's see if Linear Regression makes it any better:

In [ ]:
from sklearn.linear_model import Lasso

#SGDRegressor is for when one is adding more data as we go. Linear Regresion would be a better fit for this problem.
#Make sure to regularize. Make sure paramter used to regularize is tested in gridsearch.
#Regularization for regression. Adds penalties
#Ridge is generally used for regularization (method of linear regression to reduce overfitting. Adding term of cost function)
#R^2 Directly related to RMSE but takes out relation to units we're using. Need to pair with normalization of data.
#alpha = regularization term
#Check best_params for gscv

model = Lasso()
params={'alpha': [2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.01]}
gscv = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_mean_squared_log_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)

gscv.fit(X_train, y_train)
#best estimators
print(gscv.best_estimator_)
#printing best scores
print(gscv.best_score_)
#printing best parameters for optimal parameter tuning
print(gscv.best_params_)



In [ ]:
test["price"] = np.expm1(predictions)
test['test_id'] = 
test[["test_id", "price"]].to_csv("kyungjunism_submission_xgb.csv", index = False)